# Example Annealing Protocol

The below function illustrates the basic operation of particle simulated annealing by implementing a simple protocol. This operates on input distrance restraints, which are specified in terms of the indices of the restrained particles and the corresponding distance limits for those restraints. The function uses the `runDynamics()` function to update the particle coordinates.

This function comes from the `nuc_dynamics.dyn_util` module (written in Cython), which is assumed to have been compiled (see the `setup_cython.py` script), compile command:

```bash
$ python setup_cython.py build_ext --inplace
```

And the `nuc_dynamics` package must can be accessible in the same directory as this script, or otherwise on the PYTHONPATH.

For full genome structure calculation from Hi-C derived restraints the `nuc_dynamics.run_dyn.anneal_genome()` should be used as this both makes the distance restraints and implements an annealing protocol.

In [1]:
import numpy as np
import nuc_dynamics.dyn_util as dyn_util

def anneal_structure(coords, restraintIndices, restraintLimits,
                    tempMax=5000.0, tempMin=10.0, tempSteps=500,
                    dynSteps=100, timeStep=0.001, randSeed=None):
    """
    A simple single-stage annealing protocol with exponential
    temperature decay and sigmoidal repulsion scheme.
    """

    from math import atan, log, exp, pi

    # Consider masses and fixed particles

    if randSeed is None:
        np.random.seed()
    else:
        np.random.seed(randSeed)

    # Ensure inputs are numpy.ndarray  
    coords = np.array(coords)
    restraintIndices = np.array(restraintIndices, dtype=np.int32)
    restraintLimits = np.array(restraintLimits)

    nPoints = len(coords)

    # Masses and radii not used, just set to 1.0
    masses = radii = np.ones(nPoints, float)

    # Ambiguiuty not used, but represents the stride of sequential, grouped restraints
    ambiguity = np.ones(len(restraintIndices), np.int32)

    adj = 1.0 / atan(10.0)
    decay = log(tempMax/tempMin)    

    for step in range(tempSteps):
        frac = step/float(tempSteps)

    # exponential temp decay
    temp = tempMax * exp(-decay*frac)

    # sigmoidal repusion scheme
    repulse = 0.5 + adj * atan(frac*20.0-10) / pi 

    # update coordinates
    dyn_util.runDynamics(coords, masses, radii, restraintIndices, restraintLimits,
                         ambiguity, temp, timeStep, dynSteps, repulse,
                         repDist=1.5, printInterval=np.int32(100))

    # Center
    coords -= coords.mean(axis=0)

    return coords


The below code tests the above function by performing a small, quick calculation and demonstrates how restraints are passed into the
function as particle indices and restraint distances.


In [2]:
from numpy import random

# Random particle coordinates - 20 test points of (x,y,z) 
nCoords = 20
coords = random.uniform(-1.0, 1.0, (nCoords,3)) 
  
# Each pair represents a connection between a pair of particles
# Restraint indices refer to indices in coordinate array
rIndices = [(0, 1), (0, 5), (0, 18), (1, 0), (1, 2),
            (1, 9), (2, 1), (2, 3), (2, 10), (3, 2),
            (3, 4), (3, 17), (4, 3), (4, 5), (4, 11),
            (5, 0), (5, 4), (5, 12), (6, 7), (6, 17),
            (6, 19), (7, 6), (7, 14), (7, 15), (7, 16),
            (8, 18), (9, 1), (10, 2), (11, 4), (12, 5),
            (13, 17), (14, 7), (15, 7), (16, 7), (17, 3),
            (17, 6), (17, 13), (18, 0), (18, 8), (19, 6)]
  
# Distances - upper and lower bounds for each particle
# Unform upper and lower bounds for test restraints
lower = 0.8
upper = 1.2
rDists = [(0.8, 1.2)] * len(rIndices)

# Run calculation
coords = anneal_structure(coords, rIndices, rDists)
  
# Show results
for i, c in enumerate(coords):
    print('%3d' % i, c)

True
temp: 175.73  fRep: 684.84  fDist: 181.12  rmsd:   0.45  nViol:   36  nRep:  171
  0 [-2.27993345 -0.27495546 -2.01477024]
  1 [-1.17992727  0.23135625 -1.79642532]
  2 [-0.4277382   0.4016884  -0.82769575]
  3 [-0.15514722 -0.18461584  0.23661993]
  4 [-0.52558562 -0.92700727 -0.3304348 ]
  5 [-1.77327788 -0.03121563 -0.51654587]
  6 [1.23330654 0.82696008 1.92348844]
  7 [ 2.39040303 -0.04492806  1.93839945]
  8 [-1.45809453 -1.67537762 -3.06586828]
  9 [-1.32246126  1.57956164 -2.53708711]
 10 [ 0.24904795 -0.52355067 -2.12247142]
 11 [-1.34428605 -2.0454331   0.05885367]
 12 [-2.37631598  1.54516916 -0.6130243 ]
 13 [ 0.47664827 -0.90717234  2.26553807]
 14 [ 2.54611502 -0.39641611  0.83080719]
 15 [3.46892715 0.38228828 3.07821583]
 16 [ 2.33861845 -0.29652143  3.19424711]
 17 [ 1.20573243 -0.10416941  1.0264474 ]
 18 [-2.10190829 -0.33951042 -3.23306659]
 19 [1.03587694 2.78384954 2.5047726 ]


# Genome Structure Calculation

Although whole genome structure calculations for Hi-C contact data may be convieniently calculated using the `nuc_dynamics.sh` script, all of the functionality is also available in the `nuc_dynamics` Python module. The below example illustrates how to access the functions from this module to run a genome calculation from a Python script, though we don't recommend using this Jupyter notebook to run long calculations.

In [3]:
from time import time

def demo_calc_genome_structure():
    """
    Example of settings for a typical genome structure calculation from input single-cell
    Hi-C contacts stored in an NCC format file (as output from NucProcess)
    """

    # Import the nuc_dynamics module, which whold be in the same directory as this script
    # or otherwise on the PYTHONPATH
    from nuc_dynamics.main import calc_genome_structure

    # Set the input NCC file path (containing contact data) and the output PDB format file 
    ncc_file_path = 'example_chromo_data/Cell_1_contacts.ncc.gz'
    pdb_file_path = 'example_chromo_data/Cell_1_structure.pdb'

    # Number of alternative conformations to generate from repeat calculations
    # with different random starting coordinates
    num_models = 2

    # Parameters to setup restraints and starting coords
    general_calc_params = {'dist_power_law':-0.33,
                           'contact_dist_lower':0.8, 'contact_dist_upper':1.2,
                           'backbone_dist_lower':0.1, 'backbone_dist_upper':1.1,
                           'random_seed':int(time()), 'random_radius':10.0}

    # Annealing & dyamics parameters: the same for all stages
    # (this is cautious, but not an absolute requirement)
    anneal_params = {'temp_start':5000.0, 'temp_end':10.0, 'temp_steps':500,
                     'dynamics_steps':100, 'time_step':0.001}

    # Hierarchical scale protocol: calculations will initially use 8 Mb particles
    # but deminish to 100 kb at the end. The whole annealing protocol (hot to cold)
    # will be run at each size, but subsequent stages will start from the previous
    # structure
    particle_sizes = [8e6, 4e6, 2e6, 4e5, 2e5, 1e5]

    # Contacts must be clustered with another within this separation threshold
    # (at both ends) to be considered supported, i.e. not isolated
    # This removes noise contacts
    isolation_threshold=2e6

    # Actually run the calculation with the specified parameters and input
    # The below function will automatically create the appropriate distance restraints
    calc_genome_structure(ncc_file_path, pdb_file_path, general_calc_params, anneal_params,
                          particle_sizes, num_models, isolation_threshold)


The test function can be run with:

In [ ]:
demo_calc_genome_structure()